In [87]:
import requests
import openai
from utils.ln_oauth import auth, headers, read_creds

In [54]:
def __user_info(headers):
    '''
    Get user information from Linkedin
    '''
    response = requests.get('https://api.linkedin.com/v2/me', headers = headers)
    user_info = response.json()
    return user_info

In [96]:
def generate_post(prompt, openai):

    model = "text-davinci-003"

    completions = openai.Completion.create(
        engine=model,
        prompt=prompt,
        max_tokens=2048,
        n=1,
        stop=None,
        temperature=0.9,
    )

    # Get the first generated content
    generated_text = completions.choices[0].text
    return generated_text   

In [93]:
def post_linkedin(access_token, prompt, headers):
    headers = headers(access_token) # Make the headers to attach to the API call.
    user_info = __user_info(headers) # Get user info

    urn = user_info['id']
    author = f'urn:li:person:{urn}'
    api_url = 'https://api.linkedin.com/v2/ugcPosts'

    message = prompt+"\n\n**Este post foi gerado pelo ChatGPT como um experiemento**"

    post_data = {
        "author": author,
        "lifecycleState": "PUBLISHED",
        "specificContent": {
            "com.linkedin.ugc.ShareContent": {
                "shareCommentary": {
                    "text": message
                },
                "shareMediaCategory": "NONE"
            }
        },
        "visibility": {
            "com.linkedin.ugc.MemberNetworkVisibility": "PUBLIC"
        }
    }

    r = requests.post(api_url, headers=headers, json=post_data)
    
    return r.json()

In [ ]:
credentials = 'credentials.json'
credentials_dict = read_creds(credentials)
access_token = auth(credentials) # Authenticate the API

# Get OpenAI - API key
openai.api_key = credentials_dict['openai_access_token']

In [101]:
# What you are asking for GPT-3 to create
prompt = """
Imagine que voce é um cientista de dados senior, e quer começar a postar no linkedin de forma difundir conhecimento sobre o mundo de ciencia de dados
Voce quer falar sobre temas ou tecnologias relevante para o mundo de ciencia de dados.
Gere um post
"""

In [103]:
generated_text = generate_post(prompt, openai)
print(generated_text)


Recentemente, tem havido um grande aumento da adoção de tecnologias de aprendizado de máquina. Estas tecnologias têm sido utilizadas em uma ampla gama de domínios, desde a agricultura até o marketing digital. Como cientista de dados senior, eu tenho sentido a responsabilidade de disseminar informações sobre como essas tecnologias podem ser usadas para solucionar oportunidades de negócios. Hoje compartilho algumas dicas para ajudar quem quer começar a usar aprendizado de máquina:

• Entenda os problemas específicos que você quer resolver antes de começar a usar aprendizado de máquina.
• Examine os dados que você tem à sua disposição para testar seu modelo de aprendizado de máquina.
• Comece com algoritmos simples, como regressão linear, para entender melhor o comportamento dos dados e os padrões que ocorrem neles.
• Utilize técnicas de validação cruzada e avaliação para escolher o melhor modelo para os dados.
• Use as métricas apropriadas para medir o desempenho do seu modelo.
• Teste 

In [ ]:
post_linkedin(access_token, generated_text, headers)